In [7]:
dirmpl8='/Users/astro/Documents/notebooks/manga/spectro/analysis/MPL-8/'
resmpl8='/Users/astro/Documents/notebooks/zs/result/pair/'

In [55]:
ndis=60
delv=500
time='2019-05-08-14:35:54'

In [4]:
pairfile=dirmpl8+'drpall-v2_5_3.fits'

In [56]:
import numpy as np
from astropy import units as u
from astropy.coordinates import Distance, SkyCoord
from astropy.io import fits
from astropy.table import Table


def pair(plateifu, redshift, mangaid, ramanga, decmanga, vel,f1):
    pair = []
    array = []
    for i in range(0, len(plateifu), 1):
        z1 = redshift[i]
        if z1 >= 0:
            # primary的信息:
            ifu_p = plateifu[i]
            ra_p = ramanga[i]
            dec_p = decmanga[i]
            id_p = mangaid[i]
            vel_p = vel[i]
            d_p = Distance(z=z1, unit=u.kpc, allow_negative=False)
            coord_p = SkyCoord(ra_p * u.deg, dec_p * u.deg)
            # catalog里除了primary的所有信息
            ifu_c = np.delete(plateifu, [i])
            ra_c = np.delete(ramanga, [i])
            dec_c = np.delete(decmanga, [i])
            id_c = np.delete(mangaid, [i])
            vel_c = np.delete(vel, [i])
            catalog = SkyCoord(ra=ra_c * u.deg, dec=dec_c * u.deg)
            # 计算和primary的距离
            dis = coord_p.separation(catalog).radian
            dis_cp = dis * d_p
            # 计算相对速度小于1000km/s
            vel_cp = [abs(vel_p - n) for n in vel_c]
            vel_cp = np.array(vel_cp)
            # 找出距离小于diskpc,速度小于delvkm/s的源
            catalogmsk =(dis_cp <ndis  * u.kpc) & (vel_cp < 500)
            t = np.where(catalogmsk == True)
            # pair 源的信息:
            ifu_s = ifu_c[catalogmsk]
            ra_s = ra_c[catalogmsk]
            dec_s = dec_c[catalogmsk]
            id_s = id_c[catalogmsk]
            vel_s = vel_c[catalogmsk]
            dis_cps = dis_cp[catalogmsk]
            vel_cps = vel_cp[catalogmsk]
            # 收集pair_galaxyy信息,防止重复
            if ifu_s.size > 0:
                #             print(ifu_p,ifu_s)
                # 1.去掉pair_s中重复观测的像:
                # s 中没有重复的像:
                if len(id_s) == len(set(id_s)):
                    for j in range(0, len(id_s)):
                        if id_p == id_s[j]:
                            print('id_p == id_s[j]', id_p, id_s[j], ra_p,
                                  ra_s[j], dec_p, dec_s[j], vel_p, vel_s[j])
                        else:
                            p = [ifu_p, ifu_s[j]]
                            p_t = [ifu_s[j], ifu_p]
                            if p not in pair and p_t not in pair and vel_cps[
                                    j] > 0:
                                pair.append(p)
                                print(ifu_p,
                                      ifu_s[j],
                                      id_p,
                                      id_s[j],
                                      ra_p,
                                      dec_p,
                                      ra_s[j],
                                      dec_s[j],
                                      vel_p,
                                      vel_s[j],
                                      vel_cps[j],
                                      dis_cps[j],
                                      file=f1)

                else:
                    # s中有重复的像
                    # 先收集相同的的想的信息:
                    # 1 means remove the same mangid
                    ifu_s1 = []
                    id_s1 = []
                    vel_s1 = []
                    ra_s1 = []
                    dec_s1 = []
                    dis_cps1 = []
                    vel_cps1 = []
                    for j in set(id_s):
                        index = np.where(id_s == j)[0]
                        ifu_s1.append(ifu_s[index[0]])
                        id_s1.append(id_s[index[0]])
                        vel_s1.append(vel_s[index[0]])
                        ra_s1.append(ra_s[index[0]])
                        dec_s1.append(dec_s[index[0]])
                        dis_cps1.append(dis_cps[index[0]])
                        vel_cps1.append(vel_cps[index[0]])
                    for m in range(0, len(id_s1)):
                        if id_p == id_s1[m]:
                            print('id_p==id_s1[m]', id_p, id_s1[m], ra_p,
                                  ra_s1[m], dec_p, dec_s1[m], vel_p, vel_s1[m])
                        else:
                            p = [ifu_p, ifu_s1[m]]
                            p_t = [ifu_s1[m], ifu_p]
                            if p not in pair and p_t not in pair and vel_cps1[
                                    m] > 0:
                                pair.append(p)
                                print(ifu_p,
                                      ifu_s1[m],
                                      id_p,
                                      id_s1[m],
                                      ra_p,
                                      dec_p,
                                      ra_s1[m],
                                      dec_s1[m],
                                      vel_p,
                                      vel_s1[m],
                                      vel_cps1[m],
                                      dis_cps1[m],
                                      file=f1)


f1 = open(resmpl8 + '%s_pair_mpl8_%s_%s.txt' % (time,ndis, delv), 'a+')

print('primary_ifu',
      'secondary_ifu',
      'id_pri',
      'id_sec',
      'ra_pri',
      'dec_pri',
      'ra_sec',
      'dec_sec',
      'vel_pri',
      'vel_sec',
      'vel_ps',
      'dis_ps',
      'dis_unit',
      file=f1)

if __name__ == '__main__':

    # mpl6 manga
    f = fits.open(pairfile)
    data = f[1].data
    plateifu = data.field('plateifu')
    ramanga = data.field('objra')
    decmanga = data.field('objdec')
    redshift = data.field('z')
    mangaid = data.field('mangaid')
    vel = [n * 3e5 for n in redshift]
    vel = np.array(vel)
    
    pair(plateifu,redshift,mangaid,ramanga,decmanga,vel,f1)
    
    f.close()

f1.close()



id_p == id_s[j] 1-113567 1-113567 318.026426419 318.026426419 11.3451572409 11.3451572409 5022.96 5022.96
id_p == id_s[j] 1-113469 1-113469 317.943526819 317.943526819 9.27749462963 9.27749462963 11838.51 11838.51
id_p == id_s[j] 1-635503 1-635503 318.990452837 318.990452837 9.54307564379 9.54307564379 8790.36 8790.36
id_p==id_s1[m] 1-113525 1-113525 317.430068351 317.430068351 11.3552406345 11.3552406345 5083.71 5083.71
id_p == id_s[j] 1-113379 1-113379 316.541566803 316.541566803 10.3454195236 10.3454195236 5148.33 5148.33
id_p == id_s[j] 1-113375 1-113375 316.639658795 316.639658795 10.7512221884 10.7512221884 8464.5 8464.5
id_p == id_s[j] 1-177236 1-177236 256.398930517 256.398930517 34.7464176615 34.7464176615 11765.88 11765.88
id_p == id_s[j] 1-177250 1-177250 256.664046275 256.664046275 34.5430022348 34.5430022348 9066.36 9066.36
id_p == id_s[j] 1-561017 1-561017 257.532896417 257.532896417 32.294166029 32.294166029 8894.91 8894.91
id_p == id_s[j] 1-178473 1-178473 259.673834972

id_p == id_s[j] 1-569225 1-569225 213.432186232 213.432186232 43.6624813137 43.6624813137 26800.86 26800.86
id_p == id_s[j] 1-266074 1-266074 214.060210955 214.060210955 43.9229334765 43.9229334765 41316.0 41316.0
id_p == id_s[j] 1-592881 1-592881 214.647824001 214.647824001 44.1224317199 44.1224317199 8640.0 8640.0
id_p == id_s[j] 1-569225 1-569225 213.432186232 213.432186232 43.6624813137 43.6624813137 26800.86 26800.86
id_p == id_s[j] 1-266074 1-266074 214.060210955 214.060210955 43.9229334765 43.9229334765 41316.0 41316.0
id_p == id_s[j] 1-592881 1-592881 214.647824001 214.647824001 44.1224317199 44.1224317199 8640.0 8640.0
id_p==id_s1[m] 1-558912 1-558912 166.129407835 166.129407835 42.6245544187 42.6245544187 37833.0 37833.0
id_p==id_s1[m] 1-587938 1-587938 165.795923954 165.795923954 41.7052427347 41.7052427347 9237.39 9237.39
id_p==id_s1[m] 1-256456 1-256456 166.909018652 166.909018652 42.75319133 42.75319133 17570.4 17570.4
id_p==id_s1[m] 1-256457 1-256457 166.78133818 166.781

In [57]:
name=resmpl8 + '%s_pair_mpl8_%s_%s.txt' % (time,ndis, delv)
fchan=Table.read(name,format='ascii')
fchan.write(name[:-4]+'.fits')

In [6]:
a='/home/ashley/NewDisk/pair_galaxy/m-z-manga/result/19_3_31_09:22:59_pair_%s_%s.txt' %(dis,delv)
a

'/home/ashley/NewDisk/pair_galaxy/m-z-manga/result/19_3_31_09:22:59_pair_150_1000.txt'

In [23]:
(dis_cp < dis * u.kpc) & (vel_cp < delv)

NameError: name 'dis_cp' is not defined

In [24]:
dis*u.kpc

<Quantity 50. kpc>

In [28]:
50 * u.kpc

<Quantity 50. kpc>